In [1]:
"""For further details, see, e.g.
Ghiaus, C.
Causality issue in the heat balance method for calculating the design heating and cooling load 
Energy, 2013, 50, 292 - 301

or section 2.2.1 of my PhD thesis accessible at
https://www.researchgate.net/publication/326058713_Experimental_Identification_of_Physical_Thermal_Models_for_Demand_Response_and_Performance_Evaluation
"""
from sympy import *
init_printing(use_unicode=True)

In [2]:
# Nodes without thermal capacity must be placed on the left
Ro, Ri, Cw, Ci, Aw, Ai = symbols('Ro, Ri, Cw, Ci, Aw, Ai')
R = diag(Ro, Ri)
C = diag(Cw, Ci)

In [3]:
A = Matrix([[1, 0], [-1, 1]])  # Incidence matrix
nu = 3  # Number of inputs, e.g. Outdoor temperature, Solar irradiance, Heating power
u = [True, False, True, True]  # Mask for removing temperature or heat flux sources set to zero

In [4]:
nb = R.shape[0]  # Number of branches
nn = C.shape[0]  # Number of nodes
nz = 0  # Number of nodes without thermal capacity
for i in range(nn):
    if C[i, i] == 0:
        nz += 1

In [5]:
G = R.inv()
Cc = C[nz:, nz:]
nx = Cc.shape[0]  # Number of states

In [6]:
# Build state-space model
K = -A.T @ G @ A
K11 = K[:nz, :nz]
K12 = K[:nz, nz:]
K21 = K[nz:, :nz]
K22 = K[nz:, nz:]

Kb = A.T @ G
Kb1 = Kb[:nz, :]
Kb2 = Kb[nz:, :]

As = simplify(Cc.inv() @ (-K21 @ K11.inv() @ K12 + K22))
Bs0 = Cc.inv() @ (-K21 @ K11.inv() @ Kb1 + Kb2)
Bs1 = Cc.inv() @ -K21 @ K11.inv()
Bs2 = Cc.inv()
if Bs1.is_zero:
    Bs = Matrix(BlockMatrix([[Bs0, Bs2]]).as_explicit())
else:
    Bs = Matrix(BlockMatrix([[Bs0, Bs1, Bs2]]).as_explicit())

Cs = simplify(-K11.inv() @ K12)
ny = Cs.shape[0]
Ds0 = simplify(-K11.inv() @ Kb1)
Ds1 = simplify(-K11.inv())
Ds2 = zeros(ny, nu)
Ds = Matrix(BlockMatrix([[Ds0, Ds1, Ds2]]).as_explicit())

Bs = simplify(Bs[:, u])
if not Ds.is_zero:
    Ds = simplify(Ds[:, u])

In [7]:
# Additional parameters of the input matrix
Bs2 = zeros(nx, nu)
Bs2[0, 0] = Bs[0, 0]
Bs2[0, 1] = Bs[0, 1] * Aw
Bs2[1, 1] = Bs[1, 2] * Ai
Bs2[1, 2] = Bs[1, 2]
Bs = simplify(Bs2)

In [8]:
# State matrix
print(As)

Matrix([[-(Ri + Ro)/(Cw*Ri*Ro), 1/(Cw*Ri)], [1/(Ci*Ri), -1/(Ci*Ri)]])


In [10]:
# Input matrix
print(Bs)

Matrix([[1/(Cw*Ro), Aw/Cw, 0], [0, Ai/Ci, 1/Ci]])


In [13]:
# Jacobian state matrix
for s in As.free_symbols:
    print(f"\ndA / d{s}")
    print("-" * 8)
    print(simplify(diff(As, s)))


dA / dCi
--------
Matrix([[0, 0], [-1/(Ci**2*Ri), 1/(Ci**2*Ri)]])

dA / dRi
--------
Matrix([[1/(Cw*Ri**2), -1/(Cw*Ri**2)], [-1/(Ci*Ri**2), 1/(Ci*Ri**2)]])

dA / dRo
--------
Matrix([[1/(Cw*Ro**2), 0], [0, 0]])

dA / dCw
--------
Matrix([[(Ri + Ro)/(Cw**2*Ri*Ro), -1/(Cw**2*Ri)], [0, 0]])


In [14]:
# Jacobian input matrix
for s in Bs.free_symbols:
    print(f"\ndB / d{s}")
    print("-" * 8)
    print(simplify(diff(Bs, s)))


dB / dCi
--------
Matrix([[0, 0, 0], [0, -Ai/Ci**2, -1/Ci**2]])

dB / dAw
--------
Matrix([[0, 1/Cw, 0], [0, 0, 0]])

dB / dRo
--------
Matrix([[-1/(Cw*Ro**2), 0, 0], [0, 0, 0]])

dB / dAi
--------
Matrix([[0, 0, 0], [0, 1/Ci, 0]])

dB / dCw
--------
Matrix([[-1/(Cw**2*Ro), -Aw/Cw**2, 0], [0, 0, 0]])
